In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# # # # # # # Copy and unzip from google drive to google collab
# # # # # # # Change path
# MOVIE
!cp -r "/content/drive/MyDrive/Test/LIWC/Movies_and_TV_LIWC_Processed.zip" "/content/"
!unzip "/content/Movies_and_TV_LIWC_Processed.zip" -d '/content/'

#CD
!cp -r "/content/drive/MyDrive/Test/LIWC/CDs_and_Vinyl_LIWC_Processed.zip" "/content/"
!unzip "/content/CDs_and_Vinyl_LIWC_Processed.zip" -d '/content/'

#BOOK
!cp -r "/content/drive/MyDrive/Test/LIWC/Books_LIWC_Processed.zip" "/content/"
!unzip "/content/Books_LIWC_Processed.zip" -d '/content/'

# ELEC
!cp -r "/content/drive/MyDrive/Test/LIWC/Electronics_LIWC_Processed.zip" "/content/"
!unzip "/content/Electronics_LIWC_Processed.zip" -d '/content/'

In [ ]:
 !pip -q install catboost
 !pip -q install git+https://github.com/eli5-org/eli5
 !pip -q install cmake

# #lightGBM installation
 !git clone --recursive https://github.com/microsoft/LightGBM
 %cd /content/LightGBM
 !mkdir build
 !cmake -DUSE_GPU=1
 !make -j$(nproc)
 !sudo apt-get -y install python-pip
 !sudo -H pip install setuptools pandas numpy scipy scikit-learn -U
 %cd /content/LightGBM/python-package
 !sudo python setup.py install --precompile

In [ ]:
# Category to process: CD, BOOK, ELEC, MOVIE
DATA_CATEGORY = 'CD'
HYPERPARAMETER = True # Run Hyperparameter
NORMAL_PREDICTION = True # Run normal prediction
FEATURE_IMPORTANT = True # Run and Get Feature Importance (CatBoost is the best)
ALL_CATEGORIES = True # Run for all data categories
ALL_DATA = True # Use all data split
DATA_DISTRIBUTION = True # Show data distribution for data set
FEATURE_MAJORITY_PREDICTION = True
CV_FEATURE_MAJORITY_PREDICTION = True
SHOW_PRED_RESULT = True
SEED = 42
EDA = True
CROSSVALIDATION = True

if DATA_CATEGORY == 'CD':

    DATA_NAME = 'CDs_and_Vinyl'

elif DATA_CATEGORY == 'BOOK':

    DATA_NAME = 'Books'

elif DATA_CATEGORY == 'ELEC':

    DATA_NAME = 'Electronics'

elif DATA_CATEGORY == 'MOVIE':

    DATA_NAME = 'Movies_and_TV'

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectKBest
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, make_scorer
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split, StratifiedKFold, learning_curve, learning_curve
from sklearn import model_selection, preprocessing, metrics
from sklearn.datasets import make_friedman1
from sklearn.feature_selection import RFE, RFECV
from sklearn.metrics import roc_auc_score, recall_score, precision_score, f1_score

import xgboost as xgb
from xgboost import XGBClassifier
from catboost import CatBoostClassifier, Pool
import lightgbm as lgb

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import itertools

from scipy.stats import randint as sp_randint
from scipy.stats import uniform as sp_uniform

import re
from tqdm.notebook import tqdm
from sklearn.feature_selection import mutual_info_classif
from datetime import datetime
from sklearn.model_selection import cross_val_score
import random

from imblearn.over_sampling import SMOTE

from sklearn.datasets import make_friedman1
from sklearn.feature_selection import RFE, RFECV
import ast

### Data Preprocessing After LIWC

In [ ]:
if ALL_CATEGORIES == False:
  path = '/content/' + DATA_NAME + '_LIWC_Processed.csv'

  df = pd.read_csv(path)

  import re
  df = df.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))

  print(df.shape)

else: 

    path = '/content/CDs_and_Vinyl_LIWC_Processed.csv'

    cd_df = pd.read_csv(path)

    cd_df = cd_df.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))

    print('CDs data shape: ' + str(cd_df.shape))

    path = '/content/Books_LIWC_Processed.csv'

    book_df = pd.read_csv(path)

    book_df = book_df.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))

    print('Books data shape: ' + str(book_df.shape))

    path = '/content/Electronics_LIWC_Processed.csv'

    elec_df = pd.read_csv(path)

    elec_df = elec_df.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))

    print('Electronics data shape: ' + str(elec_df.shape))

    path = '/content/Movies_and_TV_LIWC_Processed.csv'

    movie_df = pd.read_csv(path)

    movie_df = movie_df.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))

    print('Movie data shape: ' + str(movie_df.shape))

    #baseline dataset

    df =  cd_df.copy()

    print('Baseline data shape: ' + str(df.shape))

In [ ]:
# Make a copy of data from csv file
pred_df = df.copy()

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import numpy as np

def show_distribution(data, data_name):
  # Generate random data
  #data = df['total']

  # Colours for different percentiles
  perc_25_colour = 'gold'
  perc_50_colour = 'mediumaquamarine'
  perc_75_colour = 'deepskyblue'
  perc_95_colour = 'peachpuff'

  # Plot the Histogram from the random data
  fig, ax = plt.subplots(figsize=(25,13))

  '''
  counts  = numpy.ndarray of count of data ponts for each bin/column in the histogram
  bins    = numpy.ndarray of bin edge/range values
  patches = a list of Patch objects.
        each Patch object contains a Rectnagle object. 
        e.g. Rectangle(xy=(-2.51953, 0), width=0.501013, height=3, angle=0)
  '''
  counts, bins, patches = ax.hist(data, facecolor=perc_50_colour, edgecolor='gray')


  # Set the ticks to be at the edges of the bins.
  ax.set_xticks(bins.round(2))
  plt.xticks(rotation=70)

  # Set the graph title and axes titles
  plt.title('{}: Distribution of Helpful Ratio'.format(data_name), fontsize=25)
  plt.ylabel('Count', fontsize=15)
  plt.xlabel('Helpful Ratio', fontsize=2)
        
  # Calculate bar centre to display the count of data points and %
  bin_x_centers = 0.5 * np.diff(bins) + bins[:-1]
  bin_y_centers = ax.get_yticks()[1] * 0.25

  # Display the the count of data points and % for each bar in histogram
  for i in range(len(bins)-1):
    bin_label = "{0:,.0f}".format(counts[i])
    plt.text(bin_x_centers[i], bin_y_centers, bin_label, rotation=90, rotation_mode='anchor')

  # Display the graph
  plt.show()

In [ ]:
if DATA_DISTRIBUTION == True:
    j = 0

    for data_set in [cd_df, book_df, elec_df, movie_df]:

      j += 1

      if j == 1:
        data_name = 'CDs'
      elif j == 2:
        data_name = 'Books'
      elif j == 3:
        data_name = 'Electronics'
      elif j == 4:
        data_name = 'Movies and TVs'

      show_distribution(data_set['helpful_ratio'], data_name)

In [ ]:
def show_values(axs, orient="v", space=.01, decimal=4):
    def _single(ax):
        if orient == "v":
            for p in ax.patches:
                _x = p.get_x() + p.get_width() / 2
                _y = p.get_y() + p.get_height() + (p.get_height()*0.01)
                if decimal != 0: 
                  value = '{:.4f}'.format(p.get_height())
                else:
                  value = '{:.0f}'.format(p.get_height())
                ax.text(_x, _y, value, ha="center") 
        elif orient == "h":
            for p in ax.patches:
                _x = p.get_x() + p.get_width() + float(space)
                _y = p.get_y() + p.get_height() - (p.get_height()*0.5)
                value = '{:.4f}'.format(p.get_width())
                ax.text(_x, _y, value, ha="left")

    if isinstance(axs, np.ndarray):
        for idx, ax in np.ndenumerate(axs):
            _single(ax)
    else:
        _single(axs)

In [ ]:
if DATA_DISTRIBUTION == True:
      
    j = 0

    helpful = ['Unhelpful Review', 'Helpful Review']

    for data_set in [cd_df, book_df, elec_df, movie_df]:

        j += 1

        if j == 1:
          data_name = 'CDs'
        elif j == 2:
          data_name = 'Books'
        elif j == 3:
          data_name = 'Electronics'
        elif j == 4:
          data_name = 'Movies and TVs'
        

        fig, ax = plt.subplots(figsize=(35, 13))

        sns.set(font_scale=2)

        p = sns.countplot(data=data_set, x='helpfulness')

        show_values(p, decimal=0)

        ax.set(xticks=range(len(helpful)), xticklabels=[i for i in helpful])

        plt.title('{}: Helpfulness Count Distribution'.format(data_name))
        plt.xlabel('Helpful')
        plt.ylabel('Count')
    plt.show()

In [ ]:
if DATA_DISTRIBUTION == True:
      
    j = 0

    for data_set in [cd_df, book_df, elec_df, movie_df]:

        j += 1

        if j == 1:
          data_name = 'CDs'
        elif j == 2:
          data_name = 'Books'
        elif j == 3:
          data_name = 'Electronics'
        elif j == 4:
          data_name = 'Movies and TVs'
        

        fig, ax = plt.subplots(figsize=(35, 13))

        sns.set(font_scale=2)

        p = sns.countplot(data=data_set, x='helpful_ratio')

        plt.title('{}: Helpfulness Count Distribution'.format(data_name))

        plt.xlabel('Data')
        plt.ylabel('Count')
    plt.show()

In [ ]:
#Remove unwanted fields
cols = [
'Unnamed0',    # Unrelated fields
'helpful_ratio', # helpful related feature will get close to 1
'positive',      # helpful related feature will get close to 1
'total',         # helpful related feature will get close to 1
'Segment'        # generated by LIWC for parallel processing
]

pred_df = pred_df.drop(cols,axis=1).copy()

if ALL_CATEGORIES == True:
  cd_df = cd_df.drop(cols,axis=1).copy()
  book_df = book_df.drop(cols,axis=1).copy()
  elec_df = elec_df.drop(cols,axis=1).copy()
  movie_df = movie_df.drop(cols,axis=1).copy()

### EDA

### Data Frame after deleting unwanted column

In [ ]:
pred_df.info()

In [ ]:
if DATA_DISTRIBUTION == True:
  for i in range(0,140,20):
    if i == 0:
      print(pred_df.iloc[:,i:i+20].describe())
    else:
      print(pred_df.iloc[:,i+1:i+20].describe())

In [ ]:
def show_data_distribution(df_distribute, show=False):
  # import matplotlib.pyplot as plt

  # for column in pred_df:
  #   with plt.rc_context(rc={'figure.max_open_warning': 0}):
  #     plt.figure()
  #     pred_df.boxplot([column])
  if show == True:
    
    sns.set_style('dark')
    fig = plt.figure(figsize= (20,40))
    fig.subplots_adjust(hspace = 0.30, wspace = 0.30)
    k=0
    for i in range(1,34):
        ax = fig.add_subplot(11,3,i)
        sns.boxplot(x = df_distribute.iloc[:,k])
        k = k + 1
        if k == len(df_distribute.columns): break
    plt.show()

    sns.set_style('dark')
    fig = plt.figure(figsize= (20,40))
    fig.subplots_adjust(hspace = 0.30, wspace = 0.30)
    k=34
    for i in range(1,34):
        ax = fig.add_subplot(11,3,i)
        sns.boxplot(x = df_distribute.iloc[:,k])
        k = k + 1
        if k == len(df_distribute.columns): break
    plt.show()

    sns.set_style('dark')
    fig = plt.figure(figsize= (20,40))
    fig.subplots_adjust(hspace = 0.30, wspace = 0.30)
    k=68
    for i in range(1,34):
        ax = fig.add_subplot(11,3,i)
        sns.boxplot(x = df_distribute.iloc[:,k])
        k = k + 1
        if k == len(df_distribute.columns): break
    plt.show()

    sns.set_style('dark')
    fig = plt.figure(figsize= (20,40))
    fig.subplots_adjust(hspace = 0.30, wspace = 0.30)
    k=102
    for i in range(1,34):
        ax = fig.add_subplot(11,3,i)
        sns.boxplot(x = df_distribute.iloc[:,k])
        k = k + 1
        if k == len(df_distribute.columns): break
    plt.show()

In [ ]:
# Consider to loop to show data for all category
show_data_distribution(pred_df, show = DATA_DISTRIBUTION)

### Data Splitting

In [ ]:
def get_almasdi_data_split(pred_df):
  # 90 k for training
  # 45k for helpful
  # 45k for unhelpful

  # shuffle pred_df data

  pred_df = pred_df.sample(frac=1, random_state=SEED)
  pred_df = pred_df.reset_index(drop=True)

# Train data =============== Start =============================================
  train_helpful = pred_df[pred_df['helpfulness'] == 1].sample(n=45000, random_state = SEED)

  pred_df.drop(train_helpful.index,inplace = True)

  pred_df = pred_df.reset_index(drop=True)

  train_unhelpful = pred_df[pred_df['helpfulness'] == 0].sample(n=45000, random_state = SEED)

  pred_df.drop(train_unhelpful.index,inplace = True)

  pred_df = pred_df.reset_index(drop=True)

  train = pd.concat([train_helpful ,train_unhelpful])

  train = train.reset_index(drop=True)

# Train data ================= End ==========================================

# Test data =============== Start =============================================

  test_helpful = pred_df[pred_df['helpfulness'] == 1].sample(n=5000, random_state = SEED)

  test_unhelpful = pred_df[pred_df['helpfulness'] == 0].sample(n=5000, random_state = SEED)  

  test = pd.concat([test_helpful ,test_unhelpful])

  test = test.reset_index(drop=True)

# Test data ================= End =========================================

# Hyperparameter data =============== Start ===================================
  # shuffle the data
  train = train.sample(frac=1, random_state=SEED)

  X_hpo = train.sample(n=13500, random_state = SEED)

  X_hpo = X_hpo.reset_index(drop=True)
# Hyperparameter data =============== Start ===================================

  # rebuild index

  y_hpo = X_hpo['helpfulness']
  X_hpo = X_hpo.drop('helpfulness', axis=1)

  X_train = train.drop('helpfulness', axis=1)
  y_train = train['helpfulness']

  X_test = test.drop('helpfulness', axis=1)
  y_test = test['helpfulness']

  return X_train, y_train, X_test, y_test, X_hpo, y_hpo

def get_standard_data_split(pred_df, oversampling = True):

    # shuffle pred_df data
    pred_df = pred_df.sample(frac=1, random_state=SEED)
    pred_df = pred_df.reset_index(drop=True)

    X_train, X_test, y_train, y_test = train_test_split(pred_df.drop('helpfulness'\
                                  , axis=1), pred_df['helpfulness'], 
                                  test_size = 0.20, random_state=SEED)

    X_hpo, X_hpo_test, y_hpo, y_hpo_test = train_test_split(X_train, y_train, 
                                  train_size = 0.15, random_state=SEED)
    
    if oversampling == True:
        
        sm = SMOTE(random_state=SEED)
        X_train_res, y_train_res = sm.fit_resample(X_train, y_train.ravel())

        X_train = X_train_res.copy()
        y_train = y_train_res.copy()

    return X_train, y_train, X_test, y_test, X_hpo, y_hpo

## Feature Importance

In [ ]:
# Class to extend the Sklearn classifier
class SklearnHelper(object):
    def __init__(self, clf, name=None, seed=SEED, params=None):
        params['random_state'] = seed
        self.clf = clf(**params)
        self.name = name

    def train(self, x_train, y_train):
        self.clf.fit(x_train, y_train)

    def predict(self, x):
        return self.clf.predict(x)
    
    def fit(self,x,y):
        return self.clf.fit(x,y)
    
    def feature_importances(self):
        return self.clf.feature_importances_

    def cross_validation(self, X, y):
        return cross_val_score(self.clf, X, y, cv=10,scoring='accuracy', verbose = 0)

# # will show the accuracy for 0 till n feature. 
# # the feature is sorted according to their importance.
    def get_fea_imp_exp_result(self, name, tech, fea_df, fea_result_df, min, max,step):

         for i in range(min,max,step):

            feature = fea_df.loc[0:(i-1),'Feature']   
            print('Running: ' + str(name))
            print('get_fea_imp_exp_result_loop : ' + str(i))

            fea_imp_X_train = X_train[feature].copy()

            self.fit(fea_imp_X_train, y_train)

            cv_result = self.cross_validation(fea_imp_X_train, y_train)
            print('Cross Validation Data Shape: ' + str(fea_imp_X_train.shape))
            print(cv_result)

            new_value = {'model': name,
                        'technique': tech, 
                        'num_feature': feature.shape[0], 
                        'accuracy': np.mean(cv_result),
                        'f1_score': '',
                        'precision': '',
                        'recall': ''}

            fea_result_df = fea_result_df.append(new_value, ignore_index = True)

         return fea_result_df

In [ ]:
books_xg_params = {'tree_method': 'gpu_hist', 'subsample': 1.0, 'max_depth': 10, 'learning_rate': 0.3, 'gamma': 0.4, 'colsample_bylevel': 0.5, 'verbose': -1} 

books_lg_params = {'top_rate': 0.2, 'other_rate': 0.1, 'num_leaves': 31, 'learning_rate': 0.3, 'feature_fraction_bynode': 0.75, 'verbose': -1} 

books_cat_params = {'leaf_estimation_iterations': 1, 'depth': 9, 'l2_leaf_reg': 9, 'learning_rate': 0.3, 'task_type': 'GPU', 'verbose': 0}

cds_xg_params = {'tree_method': 'gpu_hist', 'subsample': 1.0, 'max_depth': 10, 'learning_rate': 0.3, 'gamma': 0.4, 'colsample_bylevel': 0.5, 'verbose': -1}

cds_lg_params = {'top_rate': 0.2, 'other_rate': 0.05, 'num_leaves': 31, 'learning_rate': 0.2, 'feature_fraction_bynode': 0.75, 'verbose': -1} 

cds_cat_params = {'leaf_estimation_iterations': 10, 'depth': 6, 'l2_leaf_reg': 6, 'learning_rate': 0.1, 'task_type': 'GPU', 'verbose': 0} 

elec_xg_params = {'tree_method': 'gpu_hist', 'subsample': 1.0, 'max_depth': 10, 'learning_rate': 0.2, 'gamma': 0, 'colsample_bylevel': 0.1, 'verbose': -1}

elec_lg_params = {'top_rate': 0.2, 'other_rate': 0.05, 'num_leaves': 31, 'learning_rate': 0.2, 'feature_fraction_bynode': 0.75, 'verbose': -1}

elec_cat_params = {'leaf_estimation_iterations': 10, 'depth': 9, 'l2_leaf_reg': 6, 'learning_rate': 0.2, 'task_type': 'GPU', 'verbose': 0}

movie_xg_params = {'tree_method': 'gpu_hist', 'subsample': 1.0, 'max_depth': 10, 'learning_rate': 0.3, 'gamma': 0.4, 'colsample_bylevel': 0.5, 'verbose': -1}

movie_lg_params =  {'top_rate': 0.6, 'other_rate': 0.3, 'num_leaves': 31, 'learning_rate': 0.2, 'feature_fraction_bynode': 1.0, 'verbose': -1} 

movie_cat_params = {'leaf_estimation_iterations': 10, 'depth': 9, 'l2_leaf_reg': 1, 'learning_rate': 0.1, 'task_type': 'GPU', 'verbose': 0}


XGBoost, LightGBM and Catboost feature importance

In [ ]:
def gen_model_fea_importance(xg, lg, cat, X_train, y_train, fea_result_df, random_state = SEED):


  xg.fit(X_train,y_train)
  lg.fit(X_train,y_train)
  cat.fit(X_train,y_train)

  # xg feature importance table =============== start =========
  xg_fea_imp = pd.DataFrame({'value':xg.feature_importances(),\
                                    'Feature':X_train.columns})

  xg_fea_imp = xg_fea_imp.sort_values(by=['value'], ascending=False)

  xg_fea_imp.reset_index(drop=True, inplace=True)

  # xg feature importance table =============== end =========

  # lg feature importance table =============== start =========
  lg_fea_imp = pd.DataFrame({'value':lg.feature_importances(),\
                                    'Feature':X_train.columns})

  lg_fea_imp = lg_fea_imp.sort_values(by=['value'], ascending=False)

  lg_fea_imp.reset_index(drop=True, inplace=True)

  # lg feature importance table =============== end =========

  # cat feature importance table =============== start =========
  cat_fea_imp = pd.DataFrame({'value':cat.feature_importances(),\
                                    'Feature':X_train.columns})

  cat_fea_imp = cat_fea_imp.sort_values(by=['value'], ascending=False)

  cat_fea_imp.reset_index(drop=True, inplace=True)
  # cat feature importance table =============== end =========

  # Feature importance with XGBoost 

  print('Start Embedded - XGBoost')

  fea_result_df = xg.get_fea_imp_exp_result(name='XGBoost', tech='XGBoost', \
                                                fea_df=xg_fea_imp,
                                                fea_result_df = fea_result_df,
                                                min=9,max=161,step=10)

  fea_result_df = lg.get_fea_imp_exp_result(name='LightGBM', tech='XGBoost', \
                                                fea_df=xg_fea_imp,
                                                fea_result_df = fea_result_df,
                                                min=9,max=161,step=10)

  fea_result_df = cat.get_fea_imp_exp_result(name='CatBoost', tech='XGBoost', \
                                                fea_df=xg_fea_imp,
                                                fea_result_df = fea_result_df,
                                                min=9,max=161,step=10)
  # Feature importance with LightGBM

  print('Start Embedded - LightGBM')

  fea_result_df = xg.get_fea_imp_exp_result(name='XGBoost', tech='LightGBM', \
                                                fea_df=lg_fea_imp,
                                                fea_result_df = fea_result_df,
                                                min=9,max=161,step=10)

  fea_result_df = lg.get_fea_imp_exp_result(name='LightGBM', tech='LightGBM', \
                                                fea_df=lg_fea_imp,
                                                fea_result_df = fea_result_df,
                                                min=9,max=161,step=10)

  fea_result_df = cat.get_fea_imp_exp_result(name='CatBoost', tech='LightGBM', \
                                                fea_df=lg_fea_imp,
                                                fea_result_df = fea_result_df,
                                                min=9,max=161,step=10)
  # Feature importance with CatBoost
  print('Start Embedded - CatBoost')
  fea_result_df = xg.get_fea_imp_exp_result(name='XGBoost', tech='CatBoost', \
                                                fea_df=cat_fea_imp,
                                                fea_result_df = fea_result_df,
                                                min=9,max=161,step=10)

  fea_result_df = lg.get_fea_imp_exp_result(name='LightGBM', tech='CatBoost', \
                                                fea_df=cat_fea_imp,
                                                fea_result_df = fea_result_df,
                                                min=9,max=161,step=10)

  fea_result_df = cat.get_fea_imp_exp_result(name='CatBoost', tech='CatBoost', \
                                                fea_df=cat_fea_imp,
                                                fea_result_df = fea_result_df,
                                                min=9,max=161,step=10)

  fea_result_df['num_feature'] = fea_result_df['num_feature'].astype(int)

  return xg_fea_imp, lg_fea_imp, cat_fea_imp, fea_result_df

Mutual Information

In [ ]:
def gen_mutual_information(X_train, y_train, xg, lg, cat, fea_result_df, random_state = SEED):

  print('Running MI')
      
  mutual_info = mutual_info_classif(X_train, y_train, random_state = SEED)

  mutual_info = pd.Series(mutual_info)
  mutual_info.index = X_train.columns

  mi_fea_imp = pd.DataFrame({'value':mutual_info,\
                                    'Feature':mutual_info.index})

  mi_fea_imp = mi_fea_imp.sort_values(by=['value'], ascending=False)

  mi_fea_imp.reset_index(drop=True, inplace=True)

  # Feature importance 3 model
  print('Running MI Prediction: XGBoost')
  fea_result_df = xg.get_fea_imp_exp_result(name='XGBoost', tech='Mutual Information', \
                                                fea_df=mi_fea_imp,
                                                fea_result_df = fea_result_df,
                                                min=9,max=161,step=10)

  print('Running MI Prediction: LightGBM')
  fea_result_df = lg.get_fea_imp_exp_result(name='LightGBM', tech='Mutual Information', \
                                                fea_df=mi_fea_imp,
                                                fea_result_df = fea_result_df,
                                                min=9,max=161,step=10)

  print('Running MI Prediction: catBoost')
  fea_result_df = cat.get_fea_imp_exp_result(name='CatBoost', tech='Mutual Information', \
                                                fea_df=mi_fea_imp,
                                                fea_result_df = fea_result_df,
                                                min=9,max=161,step=10)
  
  return mi_fea_imp, fea_result_df, mutual_info

Permutation Feature Importance

In [ ]:
def permutation_importance(model, X_val, y_val, seed=42, threshold=0.0005,
                           minimize=True, verbose=True,):
    results = {}

    res_fea = pd.DataFrame()
    y_pred = model.predict(X_val)
    
    results['base_score'] = accuracy_score(y_val,y_pred)
    if verbose:
        print(f'Base score {results["base_score"]:.5}')

    
    for col in tqdm(X_val.columns):
        freezed_col = X_val[col].copy()

        X_val[col] = np.random.RandomState(seed = seed).permutation(X_val[col])
        preds = model.predict(X_val)
        results[col] = accuracy_score(y_val,preds)

        X_val[col] = freezed_col
        
        if verbose:
            print(f'column: {col} - {results[col]:.5}')

        # Provide the permutation inforamtion ()
        new_value = {
            'value': results[col],
            'Feature': col, 
        }

        res_fea = res_fea.append(new_value, ignore_index = True)
    
    if minimize:
        bad_features = [k for k in results if results[k] > results['base_score'] - threshold]
    else:
        bad_features = [k for k in results if results[k] < results['base_score'] + threshold]
    bad_features.remove('base_score')
    
    return res_fea, results, bad_features

In [ ]:
def gen_permutation_importance(X_train, y_train, xg, lg, cat, fea_result_df, SEED):

  xg.fit(X_train, y_train)

  # the lowest the accuracy, the more important the field
  res_fead, results, bad_features = permutation_importance(model=xg,
                                                  X_val=X_train,
                                                  y_val=y_train,
                                                  verbose=False,
                                                  seed = SEED)

  pi_fea_imp = res_fead.sort_values(by=['value'], ascending=True)

  pi_fea_imp.reset_index(drop=True, inplace=True)

  # Feature importance with 3 model 
  fea_result_df = xg.get_fea_imp_exp_result(name='XGBoost', tech='Permutation Feature Importance', \
                                                fea_df=pi_fea_imp,
                                                fea_result_df = fea_result_df,
                                                min=9,max=161,step=10)

  fea_result_df = lg.get_fea_imp_exp_result(name='LightGBM', tech='Permutation Feature Importance', \
                                                fea_df=pi_fea_imp,
                                                fea_result_df = fea_result_df,
                                                min=9,max=161,step=10)

  fea_result_df = cat.get_fea_imp_exp_result(name='CatBoost', tech='Permutation Feature Importance', \
                                                fea_df=pi_fea_imp,
                                                fea_result_df = fea_result_df,
                                                min=9,max=161,step=10)
  
  return pi_fea_imp, fea_result_df

### Combination

In [ ]:
def get_column_to_delete(df, mutual_info):
  mi_0 = mutual_info[df['level_0']]
  mi_1 = mutual_info[df['level_1']]

  if mi_0 < mi_1:
    return [mutual_info[df['level_0']], mutual_info[df['level_1']], df['level_0']]
  else:
    return [mutual_info[df['level_0']], mutual_info[df['level_1']], df['level_1']]

In [ ]:
def gen_com_fea_importance(X_train, y_train, xg, lg, cat, fea_result_df, mutual_info, SEED):
  # ================== find out correlation more than 0.7 ==== Start ================
  df_corr = X_train.copy()

  cor_matrix = df_corr.corr(method='spearman').abs()

  #get only the top triangle 
  upper_tri = cor_matrix.where(np.triu(np.ones(cor_matrix.shape),k=1).astype(np.bool))

  to_drop = [column for column in upper_tri.columns if any(upper_tri[column] >= 0.7)]

  corr_matrix = df_corr.corr().abs()

  # Get field and their respective correlation
  #the matrix is symmetric so we need to extract upper triangle matrix without diagonal (k = 1)

  sol = (corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
                    .stack()
                    .sort_values(ascending=False))

  # Only equal or more than 0.7 will be considered for deletion
  df_corr = pd.DataFrame(sol[sol.values > 0.7])

  # get index as part of the field in data frame
  df_corr.reset_index(inplace=True) 

  # ================== find out correlation more than 0.7 ==== End ================

  # ======== Delete field pairing with more than 0.7 based on MI ==== Start =======

  # Get Mutual information

  # Drop the field with colleration > 0.7 based on mutual information
  df_columns = df_corr.apply(lambda x: get_column_to_delete(x, mutual_info), axis=1)

  df_corr[['level_0_mi', 'level_1_mi', 'column_to_delete']] = pd.DataFrame(df_columns.values.tolist())

  column_to_delete = df_corr['column_to_delete']

  column_to_delete = column_to_delete.unique()

  print('Column to be remove  based on correlation and mutual information')
  print(column_to_delete.size)

  X_train_reduced = X_train.drop(column_to_delete,axis=1).copy()

  print('Train data with feature reduced')
  X_train_reduced.info()

   # ======== Delete field pairing with more than 0.7 based on MI ==== end =======

  # Find the permutation important again after deletion of 0.7 pairing based on MI

  xg.fit(X_train_reduced, y_train)

  # the lowest the accuracy, the more important the field
  res_fead, results, bad_features = permutation_importance(model=xg,
                                                  X_val=X_train_reduced,
                                                  y_val=y_train,
                                                  verbose=False,
                                                  seed = SEED)

  # sort based on permutation importance
  # sorted Asecnding order, the lower the accuracy, the more important the feature 
  com_fea_imp = res_fead.sort_values(by=['value'], ascending=True)

  com_fea_imp.reset_index(drop=True, inplace=True)

  fea_result_df = xg.get_fea_imp_exp_result(name='XGBoost', tech='Combination', \
                                                fea_df=com_fea_imp,
                                                fea_result_df = fea_result_df,
                                                min=9,max=161,step=10)

  fea_result_df = lg.get_fea_imp_exp_result(name='LightGBM', tech='Combination', \
                                                fea_df=com_fea_imp,
                                                fea_result_df = fea_result_df,
                                                min=9,max=161,step=10)

  fea_result_df = cat.get_fea_imp_exp_result(name='CatBoost', tech='Combination', \
                                                fea_df=com_fea_imp,
                                                fea_result_df = fea_result_df,
                                                min=9,max=161,step=10)
  
  return com_fea_imp, fea_result_df

In [ ]:
def gen_rfe_importance(X_train, y_train,fea_result_df, min, max, step, rfe_fea_imp_df, SEED = SEED):

    xg = xgb.XGBClassifier (random_state=SEED, ctree_method= 'gpu_hist',
                tree_method = 'gpu_hist', verbose = -1)
    lg = lgb.LGBMClassifier (random_state=SEED,device = "gpu", verbose = -1, n_estimators = 100)
    cat = CatBoostClassifier (random_seed=SEED,task_type = 'GPU', silent = True, \
                              eval_metric='Accuracy',early_stopping_rounds=3, iterations = 100)
    
    j = 0

    for model in [xg, lg, cat]:
      j += 1

      if j == 1: 
          model_name = 'XGBoost'
          print('Start RFE - XGBoost')
      elif j == 2: 
          model_name = 'LightGBM'
          print('Start RFE - LightGBM')
      elif j == 3: 
          model_name = 'CatBoost'
          print('Start RFE - CatBoost')

      for i in range(min,max,step):

          print('Number of i: ' + str(i))

          selector = RFE(model, n_features_to_select=i, step=10)

          selector = selector.fit(X_train, y_train)

          features = X_train.columns[selector.support_]

          new_feature = { 'model':model_name,
              'no_feature' : selector.n_features_,
              'name_of_selected_feature': features,
              'rank_of_each_feature': selector.ranking_,
              'support_of_each_feature': selector.support_
          }

          rfe_fea_imp_df = rfe_fea_imp_df.append(new_feature, ignore_index = True)

          cv_accuracy = cross_val_score(model, X_train[features], y_train, cv=10,scoring='accuracy', verbose = 0)

          new_value = {'model': model_name,
                     'technique': 'RFE', 
                     'num_feature': i, 
                     'accuracy': np.mean(cv_accuracy) }

          fea_result_df = fea_result_df.append(new_value, ignore_index = True)

          print(fea_result_df)

    return rfe_fea_imp_df, fea_result_df


### Result and Visualization for Feature Importance

In [ ]:
if FEATURE_IMPORTANT == True:

  xg_params = {'verbose' : -1,
               'ctree_method' : 'gpu_hist',
                'tree_method' : 'gpu_hist'
                }

  lg_params = {'verbose' : -1,
               'device': 'gpu',
               'n_estimators': 100
               }

  cat_params = {'verbose' : 0,
                'task_type':'GPU',
                'iterations': 100
                }

  xg = SklearnHelper(clf=xgb.XGBClassifier, seed=SEED, params=xg_params)
  lg = SklearnHelper(clf=lgb.LGBMClassifier, seed=SEED, params=lg_params)
  cat = SklearnHelper(clf=CatBoostClassifier, seed=SEED, params=cat_params)

  j = 0

  for i in [cd_df,book_df]:
    #empty data frame to keep result of all feature importance prediction
    fea_result_df = pd.DataFrame({
     'model':[],
      'technique': [],
     'num_feature' : [],
     'accuracy': [],
     'f1_score': [],
      'precision': [],
      'recall': []
     })
    
    rfe_fea_imp_df = pd.DataFrame({
        'model':[],
     'no_feature':[],
      'name_of_selected_feature': [],
      'rank_of_each_feature' : [],
      'support_of_each_feature': []
     })

    j += 1

    if j == 1:
      data_cato_name = 'CDs'
    elif j == 2:
      data_cato_name = 'Books'
    elif j == 3:
      data_cato_name = 'Electronics'
    elif j == 4:
      data_cato_name = 'Movie'


    if ALL_DATA == True:
      X_train, y_train, X_test, y_test, X_hpo, y_hpo = get_standard_data_split(i.copy())
    else:
      X_train, y_train, X_test, y_test, X_hpo, y_hpo = get_almasdi_data_split(i.copy())

    print('Checking pred_df: ' + str(fea_result_df.shape[0]))

    print(data_cato_name + ' : ' + str(X_train.shape))

    # #  RFE ============================== Start =======================

    now = datetime.now()

    current_time = now.strftime("%H:%M:%S")
    print("RFE Current Time =", current_time)

    rfe_fea_imp_df, fea_result_df = gen_rfe_importance(X_train, y_train,fea_result_df, 9, 160, 10,rfe_fea_imp_df,SEED = SEED)

    print('model fea result: ' + str(fea_result_df.shape[0]))
    print('rfe fea important: ' + str(rfe_fea_imp_df.shape[0]))

    file_name = '/content/drive/MyDrive/Result/' + data_cato_name + '_rfe_fea_imp.csv'
    rfe_fea_imp_df.to_csv(file_name)

    file_name = '/content/drive/MyDrive/Result/' + data_cato_name + '_fea_result_df.csv'
    fea_result_df.to_csv(file_name)

    now = datetime.now()
    current_time = now.strftime("%H:%M:%S")
    print("RFE Complete Time =", current_time)

    #  RFE ============================== End =======================

    # # #  EMBEDDED ============================== Start =======================

    now = datetime.now()
    current_time = now.strftime("%H:%M:%S")
    print("EMBEDDED Current Time =", current_time)

    xg_fea_imp, lg_fea_imp, cat_fea_imp, fea_result_df = gen_model_fea_importance \
    (xg, lg, cat, X_train, y_train, fea_result_df,random_state = SEED)

    print('model fea result: ' + str(fea_result_df.shape[0]))
    print('xg fea important: ' + str(xg_fea_imp.shape[0]))
    print('lg fea important: ' + str(lg_fea_imp.shape[0]))
    print('cat fea important: ' + str(cat_fea_imp.shape[0]))

    file_name = '/content/drive/MyDrive/Result/' + data_cato_name + '_lg_fea_imp.csv'
    lg_fea_imp.to_csv(file_name)

    file_name = '/content/drive/MyDrive/Result/' + data_cato_name + '_xg_fea_imp.csv'
    xg_fea_imp.to_csv(file_name)

    file_name = '/content/drive/MyDrive/Result/' + data_cato_name + '_cat_fea_imp.csv'
    cat_fea_imp.to_csv(file_name)

    file_name = '/content/drive/MyDrive/Result/' + data_cato_name + '_fea_result_df.csv'
    fea_result_df.to_csv(file_name)

    now = datetime.now()
    current_time = now.strftime("%H:%M:%S")
    print("EMBEDDED Complete Time =", current_time)

    # # #  EMBEDDED ============================== END =======================

    #  MI ============================== Start =======================

    now = datetime.now()
    current_time = now.strftime("%H:%M:%S")
    print("MI Start Time =", current_time)

    mi_fea_imp, fea_result_df, mutual_info = gen_mutual_information(X_train, y_train, \
                                          xg, lg, cat, fea_result_df, \
                                          random_state = SEED)

    print('MI fea result: ' + str(fea_result_df.shape[0]))

    print('MI fea important: ' + str(mi_fea_imp.shape[0]))

    file_name = '/content/drive/MyDrive/Result/' + data_cato_name + '_mi_fea_imp.csv'
    mi_fea_imp.to_csv(file_name)

    now = datetime.now()
    current_time = now.strftime("%H:%M:%S")
    print("MI Complete Time =", current_time)

    # #  MI ============================== END =======================

    # #  PI ============================== Start =======================
    now = datetime.now()
    current_time = now.strftime("%H:%M:%S")
    print("PI Start Time =", current_time)
      
    pi_fea_imp, fea_result_df = gen_permutation_importance(X_train, y_train, \
                                                  xg, lg, cat, fea_result_df, 
                                                  SEED = SEED)

    print('PI fea result: ' + str(fea_result_df.shape[0]))

    print('PI fea important: ' + str(pi_fea_imp.shape[0]))

    file_name = '/content/drive/MyDrive/Result/' + data_cato_name + '_pi_fea_imp.csv'
    pi_fea_imp.to_csv(file_name)

    now = datetime.now()
    current_time = now.strftime("%H:%M:%S")
    print("Complete Time =", current_time)

     # #  PI ============================== End =======================

    # #  COM ============================== Start =======================
    now = datetime.now()
    current_time = now.strftime("%H:%M:%S")
    print("COM Start Time =", current_time)

    com_fea_imp, fea_result_df = gen_com_fea_importance(X_train, y_train, xg, lg, \
                                            cat, fea_result_df, mutual_info, SEED = SEED)

    print('COM fea: ' + str(fea_result_df.shape[0]))

    print('COM fea important: ' + str(com_fea_imp.shape[0]))

    file_name = '/content/drive/MyDrive/Result/' + data_cato_name + '_com_fea_imp.csv'
    com_fea_imp.to_csv(file_name)

    now = datetime.now()
    current_time = now.strftime("%H:%M:%S")
    print("Complete Time =", current_time)
    # #  COM ============================== End =======================

    file_name = '/content/drive/MyDrive/Result/' + data_cato_name + '_fea_result_df.csv'
    fea_result_df.to_csv(file_name)


In [ ]:
def create_fea_histo(fea_result_df, data_name):

      sns.set(font_scale=1.5)

      fea_result_df = fea_result_df[fea_result_df['Data Set'] == data_name]

      plt.figure(figsize = (20,15))
      lp = sns.lineplot(data=fea_result_df, x='num_feature', y='accuracy', hue='Model ',ci=None, marker='o')
      lp.legend(loc='upper right',bbox_to_anchor=(1.19, 1))
      plt.title('{}: Cross Validation Accuracy for All Feature Selection'.format(data_name))
      plt.xlabel('Number of Feature')
      plt.ylabel('Accuracy')
      plt.show()

In [ ]:
def read_fea_result():

      file_name = '/content/drive/MyDrive/Result/Feature_Selection_CV_All.csv'
      return pd.read_csv(file_name)

In [ ]:
if FEATURE_IMPORTANT == True:

    for i in ['CDs', 'Books' ,'Electronics', 'Movies']:
        fea_result_df = read_fea_result()

        create_fea_histo(fea_result_df, i)

## Prediction


In [ ]:
def get_prediction_result(clf, X_train, X_test, y_train, y_test, pred_result, data_name):

  clf.fit(X_train, y_train)

  y_pred = clf.predict(X_test)

  print(clf.name + ' ' + data_name + str(X_train.shape))
  print('Number of feature - ' + data_name + ' : ' + str(len(X_train.columns)) )

  new_value = {'model': clf.name,
               'data': data_name,
              'accuracy': accuracy_score(y_test, y_pred),
              'f1_score': f1_score(y_test, y_pred),
              'precision': precision_score(y_pred = y_pred, y_true = y_test),
              'recall': recall_score(y_pred = y_pred, y_true = y_test)}

  pred_result = pred_result.append(new_value, ignore_index = True)

  return pred_result

In [ ]:
def run_prediction_mode (pred_result_df, pred_df, data_cato_name,fea_imp, num_fea, xg_params, lg_params, cat_params):

  xg = SklearnHelper(clf=xgb.XGBClassifier, seed=SEED, \
                   params=xg_params, name = 'XGBoost')
  lg = SklearnHelper(clf=lgb.LGBMClassifier, seed=SEED, \
                   params=lg_params, name = 'LightGBM')
  cat = SklearnHelper(clf=CatBoostClassifier, seed=SEED, \
                    params=cat_params, name = 'CatBoost')

  #All data
  X_train_all, y_train_all, X_test_all, y_test_all, X_hpo_all, y_hpo_all = get_standard_data_split(pred_df.copy())

  # 100k - almasdi
  X_train, y_train, X_test, y_test, X_hpo, y_hpo = get_almasdi_data_split(pred_df.copy())

  features = fea_imp.loc[0:num_fea-1,'Feature']
  
  # All data
  X_train_all_fea = X_train_all[features].copy()

  print('Number of Training feature {}'.format(len(X_train_all_fea.columns)))

  X_test_all_fea = X_test_all[features].copy()

  print('Number of Test feature {}'.format(len(X_test_all_fea.columns)))

  #alma
  X_train_fea = X_train[fea_imp.loc[0:num_fea-1,'Feature']].copy()

  X_test_fea = X_test[fea_imp.loc[0:num_fea-1,'Feature']].copy()

  for i in [xg, lg, cat]:
    pred_result_df = get_prediction_result(i, X_train_all_fea, X_test_all_fea, y_train_all, \
                                          y_test_all, pred_result_df, data_cato_name + '_all')
                                          
    
    pred_result_df = get_prediction_result(i, X_train_fea, X_test_fea, y_train, \
                                          y_test, pred_result_df, data_cato_name + '_100K')

  return pred_result_df
  

In [ ]:
def process_hy_result(model, data_cato_name, best_param, best_score, hy_result_df):

  new_value = {
      'model':model,
        'data_cato_name': data_cato_name,
        'best_parameter': best_param,
        'best_score': best_score,
        }

  hy_result_df = hy_result_df.append(new_value,ignore_index = True)

  return hy_result_df

In [ ]:
def read_fea_imp(data_cato_name, tech):

    if tech == 'pi':
      file_name = '/content/drive/MyDrive/Result/' + data_cato_name + '_pi_fea_imp.csv'
      return pd.read_csv(file_name)

    elif tech == 'com':
      file_name = '/content/drive/MyDrive/Result/' + data_cato_name + '_com_fea_imp.csv'
      return pd.read_csv(file_name)

    elif tech == 'mi':
      file_name = '/content/drive/MyDrive/Result/' + data_cato_name + '_mi_fea_imp.csv'
      return pd.read_csv(file_name)

    elif tech == 'majority':
      file_name = '/content/drive/MyDrive/Result/' + data_cato_name + '_majority_fea_imp.csv'
      return pd.read_csv(file_name)

    return

In [ ]:
def create_pred_result_histo(pred_result_df):

      for i in ['Accuracy', 'Precision', 'Recall', 'F1 Score']:

          if i == 'Accuracy':
              y = 'accuracy'
          elif i == 'Precision':
              y = 'precision'
          elif i == 'Recall':
              y = 'recall'
          elif i == 'F1 Score':
              y = 'f1_score'

          fig, ax = plt.subplots(figsize=(20, 7))

          sns.set(font_scale=1.2)

          p = sns.barplot(data=pred_result_df, \
                        x="data", y=y, hue='model')
          
          show_values(p)

          plt.title('{} by Data Set and Model - {}'.format(i, data_cato_name))
          plt.xlabel('Data')
          plt.ylabel(i)
          plt.ylim(0.80, 1.00)
          plt.show()

In [ ]:
def process_cv_result(model, cv_result, data_train_test, data_all_100k, data_cato_name, cv_result_df):

  for i in cv_result:
      new_value = {
        'model': model,
        'accuracy': i,
        'data_train_test': data_train_test,
        'data_all_100k': data_all_100k,
        'dataset': data_cato_name,
        }

      cv_result_df = cv_result_df.append(new_value,ignore_index = True)

  return cv_result_df

In [ ]:
def get_cv_result(i, data_cato_name,fea_imp, num_fea,xg_params, lg_params, cat_params, cv_result_df):

    xg = SklearnHelper(clf=xgb.XGBClassifier, seed=SEED, \
                      params=xg_params, name = 'XGBoost')
    lg = SklearnHelper(clf=lgb.LGBMClassifier, seed=SEED, \
                      params=lg_params, name = 'LightGBM')
    cat = SklearnHelper(clf=CatBoostClassifier, seed=SEED, \
                        params=cat_params, name = 'CatBoost')

    #All data
    X_train_all, y_train_all, X_test_all, y_test_all, X_hpo_all, y_hpo_all = get_standard_data_split(i.copy())

    # 100k - almasdi
    X_train, y_train, X_test, y_test, X_hpo, y_hpo = get_almasdi_data_split(i.copy())

    features = fea_imp.loc[0:num_fea-1,'Feature']

    print('Number of features: {}'.format(len(features)))
      
    #only take 80 features for all data
    X_train_all_fea = X_train_all[features].copy()

    X_test_all_fea = X_test_all[features].copy()

    #only take 80 features for alma
    X_train_fea = X_train[features].copy()

    X_test_fea = X_test[features].copy()

    print('X_train_all_fe: {}, X_test_all_fea: {}, X_train_fea: {}, X_test_fea \
    {}'.format(len(X_train_all_fea.columns),len(X_test_all_fea.columns),\
               len(X_train_fea.columns),len(X_test_fea.columns)))

# XG Boost - all data ===================================================
    print('XGBoost Cross Validation Start All Data- ' + data_cato_name)
    now = datetime.now()

    current_time = now.strftime("%H:%M:%S")
    print("Current Time =", current_time)

    cv_result = xg.cross_validation(X_train_all_fea, y_train_all)

    cv_result_df = process_cv_result('XGBoost', cv_result, 'train', 'all', data_cato_name,cv_result_df)

    print('XGBoost Cross Validation End All Data- ' + data_cato_name)
# XG Boost - all data - end =============================================

# XG Boost - 100K data - Start========================================
    print('XGBoost Cross Validation Start 100K Data- ' + data_cato_name)

    now = datetime.now()

    current_time = now.strftime("%H:%M:%S")
    print("Current Time =", current_time)

    cv_result = xg.cross_validation(X_train_fea, y_train)

    cv_result_df = process_cv_result('XGBoost', cv_result, 'train', '100k', data_cato_name,cv_result_df)

    print('XGBoost Cross Validation End 100K Data- ' + data_cato_name)
# XG Boost - 100K data - End========================================

# Ligth GBM- all data - Start========================================
    print('LightGBM Cross Validation Start All Data- ' + data_cato_name)

    now = datetime.now()

    current_time = now.strftime("%H:%M:%S")
    print("Current Time =", current_time)

    cv_result =  lg.cross_validation(X_train_all_fea, y_train_all)

    cv_result_df = process_cv_result('LightGBM', cv_result, 'train', 'all', data_cato_name,cv_result_df)

    print('LightGBM Cross Validation End All Data- ' + data_cato_name)
# Ligth GBM- all data - End ========================================


# Ligth GBM- 100k data - Start========================================
    print('LightGBM Cross Validation Start 100K Data- ' + data_cato_name)

    now = datetime.now()

    current_time = now.strftime("%H:%M:%S")
    print("Current Time =", current_time)

    cv_result =  lg.cross_validation(X_train_fea, y_train)

    cv_result_df = process_cv_result('LightGBM', cv_result, 'train', '100K', data_cato_name,cv_result_df)

    print('LightGBMCross Validation End 100K Data- ' + data_cato_name)
# Ligth GBM- 100k data - End========================================

# CatBoost- all data - Start========================================
    print('CatBoost Cross Validation Start All Data- ' + data_cato_name)

    now = datetime.now()

    current_time = now.strftime("%H:%M:%S")
    print("Current Time =", current_time)

    cv_result =  cat.cross_validation(X_train_all_fea, y_train_all)

    cv_result_df = process_cv_result('CatBoost', cv_result, 'train', 'all', data_cato_name,cv_result_df)

    print('CatBoost Cross Validation End All Data- ' + data_cato_name)
# CatBoost- all data - End ========================================

# CatBoost- 100k data - Start========================================
    print('CatBoost Cross Validation Start 100K Data- ' + data_cato_name)

    now = datetime.now()

    current_time = now.strftime("%H:%M:%S")
    print("Current Time =", current_time)

    cv_result =  cat.cross_validation(X_train_fea, y_train)

    cv_result_df = process_cv_result('CatBoost', cv_result, 'train', '100k', data_cato_name,cv_result_df)

    print('CatBoost Cross Validation End 100K Data- ' + data_cato_name)
# CatBoost- 100k data - End========================================

    return cv_result_df

In [ ]:
def run_prediction_mode (X_train_all, y_train_all, X_test_all, y_test_all, X_train_100K, y_train_100K, \
                         X_test_100K, y_test_100K, name, pred_result_df, feature_selection, fea_imp_df, best_fea_hpo_df):

  if feature_selection == 'mi' or feature_selection == 'pi' or feature_selection == 'com':

    print('Start Prediction: {}'.format(feature_selection))

    for model in ['XGBoost', 'LightGBM', 'CatBoost']: 

        print('Start Prediction: {}'.format(model))

        best_setup = best_fea_hpo_df[(best_fea_hpo_df['data_cato_name'] == name) & (best_fea_hpo_df['model'] == '{}_{}'.format(model,feature_selection))]

        num_features = best_setup['Best_num_feature'].iloc[0]
        print('Best number of Feature: {}'.format(num_features))

        features = fea_imp_df.loc[0:num_features-1,'Feature']

        print('Number of Feature from feature important: {}'.format(len(features)))

        # All data
        X_train_all_fea = X_train_all[features].copy()

        print('Number of column 100K  Training feature {}'.format(X_train_all_fea.shape))

        X_test_all_fea = X_test_all[features].copy()

        print('Number of column 100K  Test feature {}'.format(X_train_all_fea.shape))

        #alma
        X_train_fea = X_train_100K[features].copy()

        X_test_fea = X_test_100K[features].copy()

        params = best_setup['best_parameter'].iloc[0]

        params = ast.literal_eval(params)

        print('Best Params: {}'.format(params))

        if model == 'XGBoost':
            clf_model = SklearnHelper(clf=xgb.XGBClassifier, seed=SEED, \
                   params=params, name = 'XGBoost')

        elif model == 'LightGBM':
            clf_model = SklearnHelper(clf=lgb.LGBMClassifier, seed=SEED, \
                   params=params, name = 'LightGBM')

        elif model == 'CatBoost':
            clf_model = SklearnHelper(clf=CatBoostClassifier, seed=SEED, \
                    params=params, name = 'CatBoost')

        pred_result_df = get_prediction_result(clf_model, X_train_all_fea, X_test_all_fea, y_train_all, y_test_all, pred_result_df, '{}_{}_{}_all'.format(name, model,feature_selection))
                                                
          
        pred_result_df = get_prediction_result(clf_model, X_train_fea, X_test_fea, y_train_100K, y_test_100K, pred_result_df, '{}_{}_{}_100K'.format(name, model,feature_selection))

        print(pred_result_df)

  elif feature_selection == 'rfe':

    print('Start Prediction: {}', feature_selection)

    for model in ['XGBoost', 'LightGBM', 'CatBoost']:

        print('Start Prediction: {}', model)

        best_setup = best_fea_hpo_df[(best_fea_hpo_df['data_cato_name'] == name) & (best_fea_hpo_df['model'] == '{}_{}'.format(model,feature_selection))]

        fea_rfe = best_setup['best_feature'].iloc[0]
                            
        for character in '[]':

            fea_rfe = fea_rfe.replace(character, '')

        fea_rfe = fea_rfe.split()

        fea_rfe = np.array([ast.literal_eval(x.title()) for x in fea_rfe])

        features = X_train_all.columns[fea_rfe]

        print(name)

        print(features)

        print('Number of features: {}'.format(features))

        # All data
        X_train_all_fea = X_train_all[features].copy()

        print('Number of shape Training feature {}'.format(X_train_all_fea.shape))

        X_test_all_fea = X_test_all[features].copy()

        print('Number of columns Test feature {}'.format(X_train_all_fea.shape))

        #alma
        X_train_fea = X_train_100K[features].copy()

        print('Number of column 100K Training feature {}'.format(X_train_fea.shape))

        X_test_fea = X_test_100K[features].copy()

        print('Number of column 100K  Training feature {}'.format(X_test_fea.shape))

        params = best_setup['best_parameter'].iloc[0]

        params = ast.literal_eval(params)

        print('Best Params: {}'.format(params))

        if model == 'XGBoost':
            clf_model = SklearnHelper(clf=xgb.XGBClassifier, seed=SEED, \
                  params=params, name = 'XGBoost')

        elif model == 'LightGBM':
            clf_model = SklearnHelper(clf=lgb.LGBMClassifier, seed=SEED, \
                  params=params, name = 'LightGBM')

        elif model == 'CatBoost':
            clf_model = SklearnHelper(clf=CatBoostClassifier, seed=SEED, \
                    params=params, name = 'CatBoost')

        pred_result_df = get_prediction_result(clf_model, X_train_all_fea, X_test_all_fea, y_train_all, y_test_all, pred_result_df, '{}_{}_{}_all'.format(name, model,feature_selection))

        print(pred_result_df)
                                                
        pred_result_df = get_prediction_result(clf_model, X_train_fea, X_test_fea, y_train_100K, y_test_100K, pred_result_df, '{}_{}_{}_100K'.format(name, model,feature_selection))

        print(pred_result_df)

  return pred_result_df

In [ ]:
import ast
import numpy as np

def hyperparameter_tuning(X, y, name, hy_result_df, feature_selection, best_fea_result_df, fea_imp_df):

  if feature_selection == 'mi' or feature_selection == 'pi' or feature_selection == 'com':

    best_num_fea = best_fea_result_df[best_fea_result_df['technique'] == feature_selection]

    print(best_num_fea)

    xg_features = best_num_fea[best_num_fea['model'] == 'XGBoost' ]['num_feature'].iloc[0]
    lgm_features = best_num_fea[best_num_fea['model'] == 'LightGBM' ]['num_feature'].iloc[0]
    cat_features = best_num_fea[best_num_fea['model'] == 'CatBoost' ]['num_feature'].iloc[0]

  elif feature_selection == 'rfe':

    best_xgb_num_fea = best_fea_result_df[(best_fea_result_df['technique'] == feature_selection) & (best_fea_result_df['model'] == 'XGBoost')]['num_feature'].iloc[0]
    best_lgm_num_fea = best_fea_result_df[(best_fea_result_df['technique'] == feature_selection) & (best_fea_result_df['model'] == 'LightGBM')]['num_feature'].iloc[0]
    best_cat_num_fea = best_fea_result_df[(best_fea_result_df['technique'] == feature_selection) & (best_fea_result_df['model'] == 'CatBoost')]['num_feature'].iloc[0]

    print(best_xgb_num_fea, best_lgm_num_fea, best_cat_num_fea)

    xgb_rfe = fea_imp_df[(fea_imp_df['no_feature'] == best_xgb_num_fea) & (fea_imp_df['model'] == 'XGBoost')]['support_of_each_feature'].iloc[0]
    lgm_rfe = fea_imp_df[(fea_imp_df['no_feature'] == best_lgm_num_fea) & (fea_imp_df['model'] == 'LightGBM')]['support_of_each_feature'].iloc[0]
    cat_rfe = fea_imp_df[(fea_imp_df['no_feature'] == best_cat_num_fea) & (fea_imp_df['model'] == 'CatBoost')]['support_of_each_feature'].iloc[0]

    for character in '[]':
        xgb_rfe = xgb_rfe.replace(character, '')
        lgm_rfe = lgm_rfe.replace(character, '')
        cat_rfe = cat_rfe.replace(character, '')

    xgb_rfe = xgb_rfe.split()
    lgm_rfe = lgm_rfe.split()
    cat_rfe = cat_rfe.split()

    xgb_rfe = np.array([ast.literal_eval(x.title()) for x in xgb_rfe])
    lgm_rfe = np.array([ast.literal_eval(x.title()) for x in lgm_rfe])
    cat_rfe = np.array([ast.literal_eval(x.title()) for x in cat_rfe])
      
# ========== XGBoost Hyperparameter ==== Start=================

  now = datetime.now()

  current_time = now.strftime("%H:%M:%S")

  print('Start XGBoost {}: {}'.format(feature_selection, name))
  print("Current Time =", current_time)

  xg_hyper_grid = {
               'max_depth': [2,3,5,7,10],
               'gamma': [0, 0.1, 0.2, 0.3, 0.4, 1.0, 1.5, 2.0],
               'learning_rate': [0.025,0.05,0.1,0.2,0.3],
               'subsample': [0.15, 0.5, 0.75, 1.0],
               'colsample_bylevel': [0.1, 0.2, 0.25, 0.5, 0.75, 1.0],
              'tree_method': ['gpu_hist'],
              }

  #Set up the random search with 5-fold cross validation

  xgb_clf = xgb.XGBClassifier(random_state = SEED, tree_method = 'gpu_hist', verbose=500)

  xgb_rand_cv = RandomizedSearchCV(estimator=xgb_clf,
             param_distributions=xg_hyper_grid,
             cv=10, n_iter=60,
             verbose = -1, random_state = SEED)
  
  if feature_selection == 'rfe':
      feature = X.columns[xgb_rfe]

      print('Number of Feature:' + str(len(feature)))

      X_fea = X[feature].copy()
  else:
      X_fea = X[fea_imp_df.loc[0:xg_features-1,'Feature']].copy()
  
  print('Number of feature - ' + name + ' : ' + str(len(X_fea.columns)))
  print('Shape: {}'.format(str(X_fea.shape)))

  xgb_rand_cv.fit(X_fea, y)

  print('XGBoost {} Best score reached for {}: {} with params: {} '.format(feature_selection, name, \
                                        xgb_rand_cv.best_score_, xgb_rand_cv.best_params_))
  
  now = datetime.now()
  current_time = now.strftime("%H:%M:%S")

  print('End XGBoost {}: {}'.format(feature_selection, name))
  print("Current Time =", current_time)

  hy_result_df = process_hy_result('XGBoost_{}'.format(feature_selection), name, xgb_rand_cv.best_params_, xgb_rand_cv.best_score_, hy_result_df)

  print(hy_result_df)

# # ========== XGBoost Hyperparameter ==== End =================

# # ========== LightGB< Hyperparameter ==== Start=================

  now = datetime.now()
  current_time = now.strftime("%H:%M:%S")

  print('Start LightGBM {}: {}'.format(feature_selection, name))
  print("Current Time =", current_time)

  lg_params = {'num_leaves': [3, 7, 15, 31], 
                'learning_rate': [0.025, 0.05, 0.1, 0.2, 0.3],
                'top_rate': [0.2, 0.4, 0.6, 0.7],
                'other_rate': [0.05, 0.1, 0.3],
                'feature_fraction_bynode': [0.1, 0.2, 0.25, 0.5, 0.75, 1.0],
               'device': 'gpu'
              }


  lgb_clf = lgb.LGBMClassifier(random_state = SEED, device = 'gpu',
                               metric='None', 
                               verbose=-1)

  # Initialize a RandomizedSearchCV object using 5-fold CV-
  lgb_cv = RandomizedSearchCV(estimator=lgb_clf, 
                             param_distributions=lg_params, cv = 10, 
                             scoring='accuracy', 
                             n_iter=60, 
                             verbose=-1, random_state = SEED) 
  
  if feature_selection == 'rfe':
      feature = X.columns[lgm_rfe]

      print('Number of Feature:' + str(len(feature)))

      X_fea = X[feature].copy()
  else:
      X_fea = X[fea_imp_df.loc[0:lgm_features-1,'Feature']].copy()
  
  print('Number of feature - ' + name + ' : ' + str(len(X_fea.columns)))
  print('Shape: {}'.format(str(X_fea.shape)))

  # Train on training data-
  lgb_cv.fit(X_fea, y)

  print('LightGBM {} Best score reached for {}: {} with params: {} '.format(feature_selection, name, \
                                        lgb_cv.best_score_, lgb_cv.best_params_))
  
  now = datetime.now()
  current_time = now.strftime("%H:%M:%S")

  print('End LightGBM {}: {}'.format(feature_selection, name))
  print("Current Time =", current_time)
  
  hy_result_df = process_hy_result('LightGBM_{}'.format(feature_selection), name, lgb_cv.best_params_, lgb_cv.best_score_, hy_result_df)

  print(hy_result_df)

## ========== LightGB< Hyperparameter ==== End =================

# ========== CatBoost Hyperparameter ==== Start =================
  now = datetime.now()
  current_time = now.strftime("%H:%M:%S")

  print('Start CatBoost {}: {}'.format(feature_selection, name))
  print("Current Time =", current_time)

  cat_model = CatBoostClassifier(random_state=SEED, task_type='GPU', 
                                 eval_metric = 'Accuracy',
                                 silent = True)

  params = {
      'leaf_estimation_iterations': [1, 10],
      "max_depth":[3, 6, 9],
      'learning_rate': [0.025, 0.05, 0.1, 0.2, 0.3],
      'l2_leaf_reg': [1, 3, 6, 9],
      'task_type': ['GPU'],
  }

  if feature_selection == 'rfe':
      feature = X.columns[cat_rfe]

      print('Number of Feature:' + str(len(feature)))

      X_fea = X[feature].copy()
  else:
      X_fea = X[fea_imp_df.loc[0:cat_features-1,'Feature']].copy()

  print('Number of feature - ' + name + ' : ' + str(len(X_fea.columns)))
  print('Shape: {}'.format(str(X_fea.shape)))

  randomized_search_result = cat_model.randomized_search(param_distributions = params,
                  X= X_fea,
                  y= y,
                  cv=10,
                  n_iter=60,
                  verbose=False,
                  partition_random_seed = SEED)
  
  print(randomized_search_result['params'])
  print(randomized_search_result['cv_results'])

  now = datetime.now()
  current_time = now.strftime("%H:%M:%S")

  print('End CatBoost {}: {}'.format(feature_selection, name))
  print("Current Time =", current_time)

  hy_result_df = process_hy_result('CatBoost_{}'.format(feature_selection), name, randomized_search_result['params'], randomized_search_result, hy_result_df)

  print(hy_result_df)

# ========== CatBoost Hyperparameter ==== End =================

  return hy_result_df

In [ ]:
if NORMAL_PREDICTION == True or HYPERPARAMETER == True or CROSSVALIDATION == True \
or FEATURE_MAJORITY_PREDICTION == True or CV_FEATURE_MAJORITY_PREDICTION == True: 

  j = 0
  for dataset in [cd_df,elec_df, movie_df, book_df]:

    pred_result_df = pd.DataFrame({
      'model':[],
      'data':[],
      'accuracy': [],
      'f1_score': [],
        'precision': [],
        'recall': []
    })

    j += 1

    if j == 1:
      data_cato_name = 'CDs'
      
    elif j == 4:
      data_cato_name = 'Books'
      
    elif j == 2:
      data_cato_name = 'Electronics'
      
    elif j == 3:
      data_cato_name = 'Movie'

    if HYPERPARAMETER == True:

      hy_result_df = pd.DataFrame({
      'model':[],
        'data_cato_name': [],
        'best_parameter': [],
        })
      
      X_train_all, y_train_all, X_test_all, y_test_all, X_hpo_all, y_hpo_all = get_standard_data_split(dataset.copy())

      for feature_selection in ['mi','pi','com','rfe']:
          file_name = '/content/drive/MyDrive/Result/best_fea_result.csv'

          best_fea_result_df = pd.read_csv(file_name)

          best_fea_result_df = best_fea_result_df[best_fea_result_df['Data Set'] == data_cato_name]

          file_name = '/content/drive/MyDrive/Result/{}_{}_fea_imp.csv'.format(data_cato_name,feature_selection)

          fea_imp_df = pd.read_csv(file_name)

          hy_result_df = hyperparameter_tuning(X_hpo_all, y_hpo_all, data_cato_name, hy_result_df, feature_selection = feature_selection, best_fea_result_df = best_fea_result_df, fea_imp_df = fea_imp_df)

      file_name = '/content/drive/MyDrive/Result/' + data_cato_name + '_hy.csv'
      hy_result_df.to_csv(file_name)

    if NORMAL_PREDICTION == True:

        # All Data
        X_train_all, y_train_all, X_test_all, y_test_all, X_hpo_all, y_hpo_all = get_standard_data_split(dataset.copy())

        # 100k - almasdi
        X_train_100K, y_train_100K, X_test_100K, y_test_100K, X_hpo_100K, y_hpo_100K = get_almasdi_data_split(dataset.copy())

        file_name = '/content/drive/MyDrive/Result/combine_best_num_best_feature_hpo.csv'

        best_fea_hpo_df = pd.read_csv(file_name)

        for feature_selection in ['mi','pi','com','rfe']:
            file_name = '/content/drive/MyDrive/Result/{}_{}_fea_imp.csv'.format(data_cato_name,feature_selection)

            fea_imp_df = pd.read_csv(file_name)

            pred_result_df = run_prediction_mode (X_train_all, y_train_all, X_test_all, y_test_all, X_train_100K, y_train_100K, \
                          X_test_100K, y_test_100K, data_cato_name, pred_result_df, feature_selection, fea_imp_df, best_fea_hpo_df)
            
            file_name = '/content/drive/MyDrive/Result/' + data_cato_name + '_pred_result.csv'
            pred_result_df.to_csv(file_name)

In [ ]:
def show_combine_pred_result_histogram():

  file_name = '/content/drive/MyDrive/Result/combine_pred_result.csv'
  pred_result = pd.read_csv(file_name)

  for j in ['CDs', 'Books','Electronics', 'Movies']:

      show_df = pred_result[pred_result['Category'] == j]

      w = 1

      for i in ['100K', 'All']:

          data_df = show_df[show_df['Data Set'] == i].copy()

          fig, ax = plt.subplots(figsize=(35, 13))

          sns.set(font_scale=1.75)

          p = sns.barplot(x = 'data', y = 'accuracy', data = data_df, hue='model')
          
          show_values(p)

          if w == 1:
             plt.title('{}: Accuracy by Model and Feature Selection Techniques of 100K Data Set'.format(j))
          else:
             plt.title('{}: Accuracy by Model and Feature Selection Techniques of All Data Set'.format(j))

          plt.xlabel('Data')
          plt.ylabel(i)
          plt.ylim(0.75, 1.00)
          plt.show()

          w += 1

In [ ]:
if SHOW_PRED_RESULT == True:
  show_combine_pred_result_histogram()